In [1]:
import pandas as pd
import json
import math
from shapely.geometry import MultiPoint
import ast

# our file structure:
# wisd hackathon 2023 (root folder)
# - /games (downloaded games from aws)
# -- /gameid
# --- {game_id}_e_t.csv (events and tracking data)
# --- {game_id}_pyball.csv (events data)
# - /metadata (downloaded metadata from aws)
# - /play_coords (this is what backend will read from)
# -- {game_id}_coords_by_player.json
# - this notebook
# i hope i wrote this in a way that makes sense!


In [2]:
game_ids = ['0042100301', '0042100304', '0042100307', '0042100313', '0042100401', '0042100404', '0042100302', '0042100305', '0042100311',
            '0042100314', '0042100402', '0042100405', '0042100303', '0042100306', '0042100312', '0042100315', '0042100403', '0042100406']

In [3]:
# dictionary mapping from game id to the joint_df for that game
# reads all pyball csvs from all games, generated by get_pyball.ipynb
shot_df_dict = {}
for game_id in game_ids:
    # read csv generated by get_pyball.ipynb, use first col as index
    joint_df = pd.read_csv(
        f'../games/{game_id}/{game_id}_pyball.csv', index_col=0)
    shot_df_dict[game_id] = joint_df[joint_df['eventType']
                                     == "SHOT"]  # RUN IF YOU ONLY WANT SHOTS
    # shot_df_dict[game_id] = joint_df # RUN IF YOU WANT ALL EVENTS

convex hull testing for one game

In [13]:
# create dictionary of all game shots+tracking data
# isolating the frames for each shot

# inner join the shots_df from joint.csv
# and the tracking data from that game
shots_tracking_df_dict = {}
for game_id in game_ids:

    # run these lines if you already have {game_id}_e_t.csv, will be faster
    joint_df = pd.read_csv(
        f"../games/{game_id}/{game_id}_e_t.csv", index_col=0)
    joint_df["awayPlayers_tracking"] = joint_df["awayPlayers_tracking"].map(
        lambda x: eval(x))
    joint_df["homePlayers_tracking"] = joint_df["homePlayers_tracking"].map(
        lambda x: eval(x))
    shots_tracking_df_dict[game_id] = joint_df[joint_df["eventType"] == "SHOT"]

    tracking_df = pd.read_json(
        f"../games/{game_id}/{game_id}_tracking.jsonl", lines=True)

    # merge and add suffixes to columns that originally belonged to events df and tracking df
    shots_tracking_df_dict[game_id] = pd.merge(
        shot_df_dict[game_id], tracking_df, on='wallClock', how='inner', suffixes=('_event', '_tracking'))

    # I used this line to create csvs to plot players, must include all events, not just shots
    shots_tracking_df_dict[game_id].to_csv(f'./games/{game_id}/{game_id}_e_t.csv')


FileNotFoundError: [Errno 2] No such file or directory: 'games/0042100301/0042100301_e_t.csv'

In [ ]:
def distance3d(p1: list, p2: list) -> float:
    x1, y1, z1 = p1
    x2, y2, z2 = p2
    return math.sqrt(math.pow(x2 - x1, 2) +
                     math.pow(y2 - y1, 2) +
                     math.pow(z2 - z1, 2) * 1.0)


def away_or_home(row):
    # did the away or home team take the shot
    if (row['playerId'] in row['homePlayers_event']):
        return 'home'
    elif (row['playerId'] in row['awayPlayers_event']):
        return 'away'
    else:
        print(f'ERROR: {row["EVENTNUM"]}')
        return None


def get_player_coords(row):
    team = row['shotmaker_team']
    for dict in row[f'{team}Players_tracking']:
        if dict['playerId'] == row['playerId']:
            return dict['xyz']


def get_defender_distances(row):
    # return a list of distances between each defender and the shotmaker
    team = row['shotmaker_team']
    shotmaker_coord = row['shotmaker_coord']
    dist_arr = []
    if team == 'away':
        for entry in row['homePlayers_tracking']:
            defender_coord = entry['xyz']
            dist_arr.append(distance3d(shotmaker_coord, defender_coord))
    elif team == 'home':
        for entry in row['awayPlayers_tracking']:
            defender_coord = entry['xyz']
            dist_arr.append(distance3d(shotmaker_coord, defender_coord))
    else:
        print(f'ERROR: {row["EVENTNUM"]}')
        return None

    return dist_arr


def calc_hull(row):
    # calculate hull area for away + home team
    home_coords = [p['xyz'] for p in row['homePlayers_tracking']]
    away_coords = [p['xyz'] for p in row['awayPlayers_tracking']]
    home_multipoint = MultiPoint(home_coords)
    home_convex_hull = home_multipoint.convex_hull
    away_multipoint = MultiPoint(away_coords)
    away_convex_hull = away_multipoint.convex_hull
    row['home_hull'] = home_convex_hull.area
    row['away_hull'] = away_convex_hull.area
    return row


def hull_distances(row):
    shot_maker = row['shotmaker_team']
    if shot_maker == 'home':
        return row['home_hull'] - row['away_hull']
    elif shot_maker == 'away':
        return row['away_hull'] - row['home_hull']
    else:
        print(f"ERROR: {row['EVENTNUM']}")


# create new columns on the shots_and_tracking_df
for game_id in game_ids:
    shots_tracking_df_dict[game_id] = shots_tracking_df_dict[game_id].apply(
        calc_hull, axis=1)
    # I decided to find if the shotmaker was on home or away first
    # then I found the shotmaker's coordinates
    # then I found the distance from all of the defenders
    shots_tracking_df_dict[game_id]['shotmaker_team'] = shots_tracking_df_dict[game_id].apply(
        lambda row: away_or_home(row), axis=1)
    shots_tracking_df_dict[game_id]['shotmaker_coord'] = shots_tracking_df_dict[game_id].apply(
        lambda row: get_player_coords(row), axis=1)
    shots_tracking_df_dict[game_id]['defense_distance'] = shots_tracking_df_dict[game_id].apply(
        lambda row: get_defender_distances(row), axis=1)
    shots_tracking_df_dict[game_id]['hull_diff'] = shots_tracking_df_dict[game_id].apply(
        lambda row: hull_distances(row), axis=1)


Crunch nums for hull diffs for home and away team

In [ ]:
game_data_df = pd.read_csv('games_with_team_names.csv')

hull_file = open("hull_info.txt", "w")
for game_id in game_ids:
    # get row containing this game's metadata
    game_data = game_data_df[game_data_df['nbaId'] == int(game_id)].iloc[0]
    
    home_name = game_data['homeTeamName']
    away_name = game_data['awayTeamName']
    hull_file.write(f"GAME NBA ID: {game_id}\n")
    hull_file.write(f"HOME TEAM: {home_name}\n")
    hull_file.write(f"AWAY TEAM: {away_name}\n")

    curr_df = shots_tracking_df_dict[game_id]
    final_score = game_data["final_score"]
    hull_file.write(f"FINAL SCORE: {away_name} {final_score} {home_name} \n")

    home_makes_df = curr_df.loc[(curr_df['shotmaker_team']
                                 == 'home') & (curr_df['HOMEDESCRIPTION'].str.contains("PTS"))]
    home_miss_df = curr_df.loc[(curr_df['shotmaker_team']
                                == 'home') & (curr_df['HOMEDESCRIPTION'].str.contains("MISS"))]
    home_make_avg_hull_diff = round(home_makes_df['hull_diff'].mean(), 2)
    home_miss_avg_hull_diff = round(home_miss_df['hull_diff'].mean(), 2)
    home_avg_hull_diff = round(curr_df['hull_diff'].mean(), 2)
    # hull_file.write(f'{home_name} avg_hull_diff: {home_avg_hull_diff}\n')
    hull_file.write(f'{home_name} make_avg_hull_diff: {home_make_avg_hull_diff}\n')
    hull_file.write(f'{home_name} miss_avg_hull_diff: {home_miss_avg_hull_diff}\n')

    away_makes_df = curr_df.loc[(curr_df['shotmaker_team']
                                 == 'away') & (curr_df['VISITORDESCRIPTION'].str.contains("PTS"))]
    away_miss_df = curr_df.loc[(curr_df['shotmaker_team']
                                == 'away') & (curr_df['VISITORDESCRIPTION'].str.contains("MISS"))]
    away_make_avg_hull_diff = round(away_makes_df['hull_diff'].mean(), 3)
    away_miss_avg_hull_diff = round(away_miss_df['hull_diff'].mean(), 3)
    away_avg_hull_diff = round(curr_df['hull_diff'].mean(), 2)
    # hull_file.write(f'{away_name} avg_hull_diff: {away_avg_hull_diff}\n')
    hull_file.write(f'{away_name} make_avg_hull_diff: {away_make_avg_hull_diff}\n')
    hull_file.write(f'{away_name} miss_avg_hull_diff: {away_miss_avg_hull_diff}\n')
    hull_file.write("\n\n")

hull_file.close()

In [ ]:
# made_dist, miss_dist: average distance from shooter to closest defensive player
# made_off_hull: avg hull area of the offensive team on makes
# miss_off_hull: avg hull area of the offensive team on misses

# globals
global_dict = {'made_dist': [],  'miss_dist': [], 'diff': [], 'made_off_hull': [], 'made_def_hull': [
], 'miss_off_hull': [], 'miss_def_hull': []}


def shot_made(row) -> bool:
    # determine if the player made the shot or not, required: row['shotmaker_team']
    description = ""
    if row['shotmaker_team'] == "away":
        description = "VISITORDESCRIPTION"
    elif row['shotmaker_team'] == "home":
        description = "HOMEDESCRIPTION"
    else:
        print(f"ERROR: {row['EVENTNUM']}")
        return
    if isinstance(row[description], str) and "MISS" in row[description]:
        return False
        # player missed the shot
    elif isinstance(row[description], str) and "PTS" in row[description]:
        return True


def calculate_aggregates(row, agg_dict):
    if shot_made(row):
        # add the distance from the shotmaker and the closest defender
        agg_dict['made_dist'].append(min(row['defense_distance']))
        if row['shotmaker_team'] == "away":
            agg_dict['made_off_hull'].append(row['away_hull'])
            agg_dict['made_def_hull'].append(row['home_hull'])
        elif row['shotmaker_team'] == "home":
            agg_dict['made_off_hull'].append(row['home_hull'])
            agg_dict['made_def_hull'].append(row['away_hull'])
    else:
        agg_dict['miss_dist'].append(min(row['defense_distance']))
        if row['shotmaker_team'] == "away":
            agg_dict['miss_off_hull'].append(row['away_hull'])
            agg_dict['miss_def_hull'].append(row['home_hull'])
        elif row['shotmaker_team'] == "home":
            agg_dict['miss_off_hull'].append(row['home_hull'])
            agg_dict['miss_def_hull'].append(row['away_hull'])


avgs = []  # list of all games' stats to create df with

for game_id in game_ids:
    game_stats = {'made_dist': [], 'miss_dist': [], 'diff': [0], 'made_off_hull': [], 'made_def_hull': [
    ], 'miss_off_hull': [], 'miss_def_hull': []}  # FOR THIS GAME ONLY
    shots_tracking_df_dict[game_id].apply(
        lambda row: calculate_aggregates(row, game_stats), axis=1)
    for k, v in game_stats.items():
        if k == 'diff':
            continue
        game_stats[k] = sum(v) / len(v)  # calculate avg of this game's stats
        global_dict[k].append(game_stats[k])
    game_stats['diff'] = game_stats['made_dist'] - game_stats['miss_dist']
    global_dict['diff'].append(game_stats['diff'])
    this_avgs = list(game_stats.values())
    this_avgs.insert(0, game_id)
    avgs.append(this_avgs)

for k, v in global_dict.items():
    global_dict[k] = sum(v) / len(v)
global_avg = list(global_dict.values())
global_avg.insert(0, "AVG")
avgs.append(global_avg)

avg_df = pd.DataFrame(avgs, columns=['game_id', 'made_dist', 'miss_dist', 'diff', 'made_off_hull',
                      'made_def_hull',  'miss_off_hull', 'miss_def_hull'])
avg_df.to_csv('avg.csv')


generate top ten most spaced out plays

In [ ]:
# sorting plays by greatest difference between the shotmaker team and the other team


def calc_coords(row, team):
    player_coords = []
    for player in row[f'{team}Players_tracking']:
        player_coords.append(player['xyz'])
    return player_coords


for game_id in game_ids:
    shots_tracking_df_dict[game_id]['home_coords'] = shots_tracking_df_dict[game_id].apply(
        lambda row: calc_coords(row, 'home'), axis=1)
    shots_tracking_df_dict[game_id]['away_coords'] = shots_tracking_df_dict[game_id].apply(
        lambda row: calc_coords(row, 'away'), axis=1)
    shots_tracking_df_dict[game_id] = shots_tracking_df_dict[game_id].sort_values(
        by='hull_diff', ascending=False)

    df = shots_tracking_df_dict[game_id].head(10)
    # df = df[['home_coords', 'away_coords', 'wallClock', 'ball', 'hull_diff']]
    df.to_csv(f'../games/{game_id}/{game_id}_top_ten.csv', index=False)


Format the data for visualization

In [9]:
# for each game, we identify the 10 best shots and then retrieve the data for the entire play
# the play is then aggregated by player and added 

for game_id in game_ids:
    e_t_df = pd.read_csv(f'../games/{game_id}/{game_id}_e_t.csv', index_col=0)
    top_ten_df = pd.read_csv(f'../games/{game_id}/{game_id}_top_ten.csv', index_col=0)

    shot_rank = 1 # rank of shot (1-10)
    top_ten_play_coords_by_player = {}
    for clock in top_ten_df['wallClock']:
        # row # of the shot in e_t_df:
        shot_index = e_t_df[(e_t_df['wallClock'] == clock) & (e_t_df['eventType'] == "SHOT")].index[0]

        # FOR RIGHT NOW, skipping the rest of the loop if the shotclock is 0, because I'm not sure why that's happening
        if e_t_df.loc[shot_index, 'shotClock_tracking'] == 0:
            continue

        # index starts when the shotclock is 24, or close to 24, i.e. when the time "starts over"
        start_index = shot_index
        while start_index > 0:
            curr_shotclock = e_t_df.loc[start_index, 'shotClock_tracking']
            prev_shotclock = e_t_df.loc[start_index - 1, 'shotClock_tracking']
            
            # when the shot clock is greater than the previous row, we know it "started over"
            if curr_shotclock > prev_shotclock:
                break
            start_index -= 1
            
        # the coordinates for each play start when the shotclock beings and end when the shot is taken
        play_coords_df = e_t_df.iloc[start_index:shot_index+1]

        play_coords_df = play_coords_df[['eventType', 'homePlayers_tracking', 'awayPlayers_tracking', 'ball', 'shotClock_tracking']]

        # string to JSON
        play_coords_df['homePlayers_tracking'] = play_coords_df['homePlayers_tracking'].apply(ast.literal_eval)
        play_coords_df['awayPlayers_tracking'] = play_coords_df['awayPlayers_tracking'].apply(ast.literal_eval)
        play_coords_df['ball'] = play_coords_df['ball'].apply(
            ast.literal_eval)

        # method to aggregate the data for visualization, organizing by player instead of by play
        def coords_by_player(team, color):
            player_dict = {}

            for _, row in play_coords_df.iterrows():
                tracking_info = row[f'{team}Players_tracking']
                
                for player in tracking_info:
                    jersey = player['jersey']
                    playerId = player['playerId']
                    xyz = player['xyz']
                    
                    # Append the coords to the list for the corresponding player
                    if (jersey, playerId) not in player_dict:
                        player_dict[(jersey, playerId)] = []
                    player_dict[(jersey, playerId)].append(xyz)

            # Converting the dictionary to a list of dictionaries for each player
            all_coords_list = [{'type': team, 'number': jersey, 'color': color, 'coords': xyz_list} for (jersey, playerId), xyz_list in player_dict.items()]
            return all_coords_list
        
        # method to aggregate the data for visualization, organizing by player instead of by play
        def ball_coords():
            xyz_list = []
            for _, row in play_coords_df.iterrows():
                tracking_info = row['ball']
                xyz_list.append(tracking_info)
            # Converting the dictionary to a list of dictionaries for each player
            all_coords_list = [{'type': 'ball', 'color': 'orange', 'coords': xyz_list}]
            return all_coords_list

        # dictionary mapping from the gameClock of the best shot to the players and their location data
        top_ten_play_coords_by_player[shot_rank] = coords_by_player('home', 'blue') + coords_by_player('away', 'red') + ball_coords()
        shot_rank +=1
    with open(f"../play_coords/{game_id}_coords_by_player.json", "w") as outfile:
        json.dump(top_ten_play_coords_by_player, outfile)
    # print("GAME", game_id, top_ten_play_coords_by_player)